# 영화 내용 요약 정보를 이용한 장르 분류

## Data Preprocessing

In [188]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from konlpy.tag import Okt

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [189]:
movie_df = pd.read_csv( 'movie_reviews.csv' )

In [190]:
movie_df.head()

,Unnamed: 0,title,story,genre
0,0,그린 북,1962년 미국 입담과 주먹만 믿고 살아가던 토니 발레롱가비고 모텐슨는 교양과 우아...,드라마
1,1,가버나움,나를 세상에 태어나게 한 부모님을 고소하고 싶어요출생기록조차 없이 살아온 어쩌면 1...,드라마
2,2,베일리 어게인,귀여운 소년 이든의 단짝 반려견 베일리는 행복한 생을 마감한다하지만 눈을 떠보니 다...,모험
3,3,주전장,일본의 인종차별 문제를 다룬 영상을 올린 후 우익들의 공격 대상이 된 일본계 미국인...,다큐멘터리
4,4,포드 V 페라리,1960년대 매출 감소에 빠진 포드는 판매 활로를 찾기 위해스포츠카 레이스를 장악한...,액션


In [191]:
movie_df.shape

(384, 4)

### 장르 코드 변수 추가

In [192]:
movie_genres = movie_df[ 'genre' ].unique().tolist()
movie_genres

['드라마',
 '모험',
 '다큐멘터리',
 '액션',
 'SF',
 '애니메이션',
 '범죄',
 '멜로/로맨스',
 '전쟁',
 '가족',
 '판타지',
 '코미디',
 '공연실황',
 '한국',
 '서부',
 '뮤지컬',
 '공포',
 '미스터리']

In [193]:
genre_code_dict = dict( ( c, i + 1 ) for i, c in enumerate( movie_genres ) )

In [194]:
genre_code_dict

{'드라마': 1,
 '모험': 2,
 '다큐멘터리': 3,
 '액션': 4,
 'SF': 5,
 '애니메이션': 6,
 '범죄': 7,
 '멜로/로맨스': 8,
 '전쟁': 9,
 '가족': 10,
 '판타지': 11,
 '코미디': 12,
 '공연실황': 13,
 '한국': 14,
 '서부': 15,
 '뮤지컬': 16,
 '공포': 17,
 '미스터리': 18}

In [237]:
max_genre_code_length = len( genre_code_dict )
max_genre_code_length

18

In [195]:
movie_df[ 'genre_code' ] = movie_df[ 'genre' ].map( lambda x : genre_code_dict[ x ] )

In [196]:
movie_df.head()

,Unnamed: 0,title,story,genre,genre_code
0,0,그린 북,1962년 미국 입담과 주먹만 믿고 살아가던 토니 발레롱가비고 모텐슨는 교양과 우아...,드라마,1
1,1,가버나움,나를 세상에 태어나게 한 부모님을 고소하고 싶어요출생기록조차 없이 살아온 어쩌면 1...,드라마,1
2,2,베일리 어게인,귀여운 소년 이든의 단짝 반려견 베일리는 행복한 생을 마감한다하지만 눈을 떠보니 다...,모험,2
3,3,주전장,일본의 인종차별 문제를 다룬 영상을 올린 후 우익들의 공격 대상이 된 일본계 미국인...,다큐멘터리,3
4,4,포드 V 페라리,1960년대 매출 감소에 빠진 포드는 판매 활로를 찾기 위해스포츠카 레이스를 장악한...,액션,4


In [197]:
movie_df.drop( movie_df.columns[ [ 0 ] ], axis = 'columns' )

,title,story,genre,genre_code
0,그린 북,1962년 미국 입담과 주먹만 믿고 살아가던 토니 발레롱가비고 모텐슨는 교양과 우아...,드라마,1
1,가버나움,나를 세상에 태어나게 한 부모님을 고소하고 싶어요출생기록조차 없이 살아온 어쩌면 1...,드라마,1
2,베일리 어게인,귀여운 소년 이든의 단짝 반려견 베일리는 행복한 생을 마감한다하지만 눈을 떠보니 다...,모험,2
3,주전장,일본의 인종차별 문제를 다룬 영상을 올린 후 우익들의 공격 대상이 된 일본계 미국인...,다큐멘터리,3
4,포드 V 페라리,1960년대 매출 감소에 빠진 포드는 판매 활로를 찾기 위해스포츠카 레이스를 장악한...,액션,4
...,...,...,...,...
379,바그다드 카페 : 디렉터스컷,황량한 사막 한가운데 자리 잡은 초라한 바그다드 카페 커피머신은 고장난지 오래고 먼...,코미디,12
380,론 서바이버,2005년 6월 28일 아프가니스탄에서 복무중인 네이비씰 대원 마커스 마이클 대니 ...,액션,4
381,국제시장,1950년대 한국전쟁 이후로부터 현재에 이르기까지 격변의 시대를 관통하며 살아온 우...,드라마,1
382,말레나,2차 대전이 한창인 햇빛 찬란한 지중해의 작은 마을 매혹적인 말레나 걸어갈 때면 어...,드라마,1


In [198]:
movie_df.shape

(384, 5)

### 학습에 사용할 변수에 대한 리스트 생성

In [199]:
movie_storys = np.array( movie_df[ 'story' ].values ) 
movie_genres = np.array( movie_df[ 'genre' ].values ) 
movie_genre_codes = np.array( movie_df[ 'genre_code' ].values )

In [200]:
max_sample_length = len( movie_storys )
print( '훈련용 샘플의 개수 : {}'.format( max_sample_count ) )

훈련용 샘플의 개수 : 384


In [201]:
max_genre_length = len( genre_code_dict )
print( '총 장르의 개수 : {}'.format( max_genre_length ) )

총 장르의 개수 : 18


In [202]:
print( '첫번째 샘플 영화의 장르 : {}'.format( movie_genres[ 0 ] ) )

첫번째 샘플 영화의 장르 : 드라마


In [203]:
print('첫번째 샘플 영화의 장르 코드 : {}'.format( genre_code_dict[ movie_genres[ 0 ] ] ) )

첫번째 샘플 영화의 장르 코드 : 1


In [204]:
movie_storys.ndim, movie_storys.shape

(1, (384,))

In [205]:
movie_genres.ndim, movie_genres.shape

(1, (384,))

In [206]:
movie_genre_codes.ndim, movie_genre_codes.shape

(1, (384,))

In [207]:
movie_storys[ :3 ]

array(['1962년 미국 입담과 주먹만 믿고 살아가던 토니 발레롱가비고 모텐슨는 교양과 우아함 그 자체인천재 피아니스트 돈 셜리마허샬라 알리 박사의 운전기사 면접을 보게 된다백악관에도 초청되는 등 미국 전역에서 콘서트 요청을 받으며 명성을 떨치고 있는 돈 셜리는위험하기로 소문난 미국 남부 투어 공연을 떠나기로 결심하고투어 기간 동안 자신의 보디가드 겸 운전기사로 토니를 고용한다거친 인생을 살아온 토니 발레롱가와 교양과 기품을 지키며 살아온 돈 셜리 박사생각 행동 말투 취향까지 달라도 너무 다른 두 사람은그들을 위한 여행안내서 그린북에 의존해 특별한 남부 투어를 시작하는데',
       '나를 세상에 태어나게 한 부모님을 고소하고 싶어요출생기록조차 없이 살아온 어쩌면 12살 소년 자인으로부터',
       '귀여운 소년 이든의 단짝 반려견 베일리는 행복한 생을 마감한다하지만 눈을 떠보니 다시 시작된 견생 2회차 아니 3회차1등 경찰견 엘리에서 찰떡같이 마음을 알아주는 소울메이트 티노까지다시 태어날 때마다 성별과 생김새 직업에 이름도 바뀌지만여전히 영혼만은 사랑 충만 애교 충만 주인바라기 베일리어느덧 견생 4회차 방랑견이 되어 떠돌던 베일리는마침내 자신이 돌아온 진짜 이유를 깨닫고 어딘가로 달려가기 시작하는데'],
      dtype=object)

In [208]:
movie_genres[ :3 ]

array(['드라마', '드라마', '모험'], dtype=object)

In [209]:
movie_genre_codes[ :3 ]

array([1, 1, 2], dtype=int64)

### 형태소 분석 / 불용어 처리

In [210]:
okt = Okt()

In [211]:
stop_words = []

with open( 'stopwords.txt', 'r', encoding = 'utf-8' ) as f:
    lines = f.readlines()
    for line in lines:
        stop_words.append( line.rstrip( '\n' ) )

print( stop_words[ : 100 ] )

['아', '휴', '아이구', '아이쿠', '아이고', '어', '나', '우리', '저희', '따라', '의해', '을', '를', '에', '의', '가', '으로', '로', '에게', '뿐이다', '의거하여', '근거하여', '입각하여', '기준으로', '예하면', '예를 들면', '예를 들자면', '저', '소인', '소생', '저희', '지말고', '하지마', '하지마라', '다른', '물론', '또한', '그리고', '비길수 없다', '해서는 안된다', '뿐만 아니라', '만이 아니다', '만은 아니다', '막론하고', '관계없이', '그치지 않다', '그러나', '그런데', '하지만', '든간에', '논하지 않다', '따지지 않다', '설사', '비록', '더라도', '아니면', '만 못하다', '하는 편이 낫다', '불문하고', '향하여', '향해서', '향하다', '쪽으로', '틈타', '이용하여', '타다', '오르다', '제외하고', '이 외에', '이 밖에', '하여야', '비로소', '한다면 몰라도', '외에도', '이곳', '여기', '부터', '기점으로', '따라서', '할 생각이다', '하려고하다', '이리하여', '그리하여', '그렇게 함으로써', '하지만', '일때', '할때', '앞에서', '중에서', '보는데서', '으로써', '로써', '까지', '해야한다', '일것이다', '반드시', '할줄알다', '할수있다', '할수있어', '임에 틀림없다']


In [212]:
vocab = {}
movie_token_storys = []

for story in movie_storys:
    sentence = okt.nouns( story )
    result = []

    for word in sentence: 
        if word not in stop_words: 
            result.append( word )
            if word not in vocab:
                vocab[ word ] = 0 
            vocab[ word ] += 1
    movie_token_storys.append( result ) 
print( movie_token_storys )

[['미국', '입담', '주먹', '토니', '발레', '롱', '비고', '모텐슨는', '교양', '자체', '천재', '피아니스트', '돈', '셜리마허샬', '알리', '박사', '운전기사', '면접', '백악관', '초청', '미국', '전역', '콘서트', '요청', '명성', '돈', '셜리', '미국', '남부', '투어', '공연', '결심', '투어', '기간', '보디가드', '겸', '운전기사', '토니', '고용', '인생', '토니', '발레', '롱', '교양', '기품', '돈', '셜리', '박사', '생각', '행동', '말투', '취향', '달라', '두', '사람', '위', '여행안내서', '그린북', '의존', '남부', '투어', '시작'], ['세상', '부모님', '출생', '기록', '어쩌면', '살', '소년', '자인'], ['소년', '단짝', '반려견', '베', '일리', '생', '마감', '눈', '다시', '시작', '견생', '경찰견', '엘리', '찰떡', '마음', '소울메이트', '티노', '다시', '성별', '생김새', '직업', '이름', '영혼', '사랑', '충만', '애교', '충만', '주인', '베', '일리', '견생', '방랑견', '베', '일리', '마침내', '진짜', '이유', '어딘가', '시작'], ['일본', '인종차별', '문제', '영상', '후', '우익', '공격', '대상', '일본', '미국인', '유튜버', '미키', '데', '자키', '일본군', '위안부', '관', '기사', '기자', '우익', '인신공격', '문제', '호기심', '안고', '이야기', '전하', '주장', '반격', '인물', '비밀', '발견', '숨', '틈', '전쟁', '시작'], ['매출', '감소', '포드', '판매', '활', '찾기', '위해', '스포츠카', '레이스', '장악', '절대', '위', '페라리', '인수', '합병', '추진', '막대', 

### 영화 줄거리 정수 인코딩 / 영화 장르 원-홧 인코딩

In [213]:
max_words = 1000
num_classes = len( genre_code_dict )

tokenizer = Tokenizer( num_words = max_words + 2, oov_token = 'OOV' ) # 상위 1000개 단어만 사용
tokenizer.fit_on_texts( movie_token_storys )

In [214]:
X = tokenizer.texts_to_sequences( movie_token_storys )
print( X )

[[72, 1, 864, 311, 346, 1, 1, 1, 1, 865, 158, 866, 55, 1, 1, 151, 1, 1, 1, 1, 72, 1, 737, 446, 1, 55, 1, 72, 1, 347, 205, 122, 347, 867, 868, 1, 1, 311, 738, 74, 311, 346, 1, 1, 1, 55, 1, 151, 63, 389, 1, 1, 390, 21, 8, 29, 1, 1, 1, 1, 347, 4], [11, 560, 1, 447, 1, 25, 64, 1], [64, 1, 1, 348, 448, 259, 1, 34, 12, 4, 1, 1, 1, 1, 24, 1, 1, 12, 1, 1, 634, 77, 288, 6, 1, 1, 1, 217, 348, 448, 1, 1, 348, 448, 60, 126, 159, 1, 4], [186, 1, 449, 739, 19, 1, 152, 391, 186, 1, 1, 1, 206, 1, 1, 1, 392, 740, 1, 1, 1, 449, 393, 1, 78, 635, 1, 1, 507, 67, 23, 450, 741, 51, 4], [1, 1, 636, 1, 1, 195, 3, 1, 1, 869, 451, 29, 1, 1, 1, 1, 1, 1, 187, 1, 1, 1, 1, 870, 349, 636, 1, 1, 1, 1, 1, 131, 508, 1, 21, 40, 1, 20, 871, 1, 1, 1, 1, 1, 1, 742, 452, 636, 637, 1, 1, 1, 1, 3, 1, 1, 1, 394, 871, 1, 1, 1, 1, 1, 738, 1, 743, 312, 46, 1, 1, 1, 91, 1, 1, 1, 638, 636, 1, 1, 1, 1, 1, 1, 1, 872, 1, 8, 1, 1, 1, 29, 1, 4, 1, 1], [350, 51, 1, 1, 1, 639, 16, 453, 25, 42, 23, 51, 351, 260, 16, 14, 42, 1, 42, 1, 1, 561

In [215]:
max_X_length = max( len( l ) for l in X )
max_X_length

593

In [218]:
X = pad_sequences( X, maxlen = max_X_length, padding = 'post' )
X

array([[ 72,   1, 864, ...,   0,   0,   0],
       [ 11, 560,   1, ...,   0,   0,   0],
       [ 64,   1,   1, ...,   0,   0,   0],
       ...,
       [350,  51, 180, ...,   0,   0,   0],
       [131,   1, 930, ...,   0,   0,   0],
       [  1, 316, 957, ...,   0,   0,   0]])

In [235]:
X.shape

(384, 593)

In [264]:
movie_genre_codes

array([ 1,  1,  2,  3,  4,  1,  1,  1,  1,  5,  1,  1,  2,  6,  7,  5,  5,
        1,  8,  9,  1,  1, 10,  4,  1,  6,  6,  4,  8,  2,  4,  1,  1,  7,
        1, 11,  8,  1,  1,  1,  1, 12,  6,  1,  1,  4,  1, 12,  3,  1,  3,
        1,  6,  8,  1, 11,  5,  4,  1,  1, 12,  4, 12,  1,  6,  2,  6,  6,
        1, 13,  6,  7,  3, 12,  1,  1,  2,  1, 14,  6,  6, 12,  1,  6,  1,
        6,  5,  6,  1,  6,  8, 12,  6,  5,  1,  1,  5, 10,  1,  1,  6,  4,
        4,  6,  1,  1,  6,  1,  3,  1,  3,  4,  6,  6, 11,  4, 13,  8,  6,
       12, 11,  6,  1,  1,  8, 10,  1,  1,  4,  1,  2,  3,  1,  1,  1,  6,
       13,  4,  1,  3,  1, 15,  1,  3,  6,  5,  5,  4,  1,  4,  1,  1, 12,
        3,  6,  6, 16, 11,  3,  1, 11,  1,  6,  1,  1,  1,  4,  5,  5,  6,
        8,  6,  6,  4,  6, 12,  4,  1, 10,  1,  6,  1,  6,  1, 16,  4,  1,
        7,  6,  3,  6, 12,  1,  4,  6,  1,  5, 15,  1,  6,  1, 10,  6,  6,
        6,  4,  3,  6, 12,  3,  3,  1,  6,  5,  1, 12,  1,  4,  1,  8,  1,
       12,  5,  2,  1,  7

In [265]:
y = to_categorical( movie_genre_codes )
y

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [266]:
y.shape

(384, 19)

###  훈련 데이터와 테스트 데이터 분리

In [251]:
( X_train, X_test, y_train, y_test ) = train_test_split( X, y, train_size = 0.8, random_state = 2)

In [252]:
print( X_train[ :5 ] ) 

[[186   1 449 ...   0   0   0]
 [  1  10 227 ...   0   0   0]
 [131 965   1 ...   0   0   0]
 [  1   1   1 ...   0   0   0]
 [ 97 269   1 ...   0   0   0]]


In [253]:
X_train.shape

(307, 593)

In [254]:
print( y_train[ :5 ] )
print( y_test[ :5 ] )

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [255]:
y_train.shape

(307, 19)

### 소프트맥스 회귀

In [256]:
model=Sequential()

model.add( Dense( max_genre_length, input_dim = max_X_length, activation = 'softmax' ) )
sgd = optimizers.SGD( lr = 0.01 )

model.compile( loss = 'categorical_crossentropy', optimizer = 'adam', metrics = [ 'accuracy' ] )
history = model.fit( X_train, y_train, batch_size = 1, epochs = 200, validation_data = ( X_test, y_test ) )

ValueError: A target array with shape (307, 19) was passed for an output of shape (None, 18) while using as loss `categorical_crossentropy`. This loss expects targets to have the same shape as the output.